In [1]:
%pip install accelerate==0.21.0  bitsandbytes==0.40.2   peft==0.5.0   transformers==4.34.0   sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for accelerate==0.21.0 from https://files.pythonhosted.org/packages/70/f9/c381bcdd0c3829d723aa14eec8e75c6c377b4ca61ec68b8093d9f35fc7a7/accelerate-0.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for bitsandbytes==0.40.2 from https://files.pythonhosted.org/packages/32/ea/75961538b08e4ed568057198717aabdebeaf6f398b7692420532e6861754/bitsandbytes-0.40.2-py3-none-any.whl.metadata
  Obtaining dependency information for peft==0.5.0 from https://files.pythonhosted.org/packages/37/1a/8d20e8704da9fa070eb909265584b960da57be1d833d550c59f50906dc5c/peft-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for transformers==4.34.0 from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.1 MB/s eta

In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/jupyter/datasphere/project/cache'

In [2]:
os.environ['TRANSFORMERS_CACHE']

'/home/jupyter/datasphere/project/cache'

In [3]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<s>system\n"


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

2024-02-07 14:38:26.579256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.float32, device_map="auto")
model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float32)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [6]:
from transformers import pipeline, Conversation

In [7]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{"role": "system", "content": system_prompt}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [8]:
inputs = ["Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Баня, водка, гармонь и лосось", "Почему трава зеленая?", "Ты кто?", "Привет!"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(prompt)
    print(output)
    print()
    print("==============================")
    print()

<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.<s>system
</s><s>user
Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Баня, водка, гармонь и лосось</s><s>bot
Однажды в небольшом российском городе произошло что-то особенное. В этот день все жители города собрались на праздник, посвященный 100-летию местной бани. Эта баня была не только местом для общения и отдыха, но и символом гордости и достоинства этого маленького городка.

Вот как выглядит эта история:

Давным-давно, когда еще не было телефонов и интернета, жители этого города собирались вместе в бане, чтобы поговорить, поужинать и поиграть на гармони. Один из самых известных музыкантов города, Иван, был знаменит своим талантом играть на гармони. Он мог создавать такие прекрасные мелодии, которые заставляли всех забывать о своих проблемах и радоваться жизни.

Однажды вечером, после долгих часов игры на гармони, Иван решил приготовить пирог с лососем для св